In [1]:
import torch 
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error

In [2]:
dataset = pd.read_csv("IBM.csv")
#dataset.head()


In [3]:
total_elements = dataset.shape[0]
trainrows = int( total_elements* 0.75)
testrows = int( total_elements * 0.25)
trainset = dataset.head(n=trainrows).iloc[:, 1:2].values
testset = dataset.tail(n=testrows).iloc[:, 1:2].values

Now lets scale the data bet 0 and 1

In [4]:
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(trainset)
training_set_scaled.shape

(189, 1)

In [5]:
X_train = []
y_train = []
for i in range(60, trainrows):
    X_train.append(training_set_scaled[i-60:i,0])
    y_train.append(training_set_scaled[i,0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [6]:
#Reshaping the X_train (Why????)
print(X_train.shape)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape)

(129, 60)
(129, 60, 1)


In [7]:
#LSTM architechure

regressor = Sequential()

regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))

regressor.add(Dropout(0.2))

#2nd layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

#3rd layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

#4th layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

#Output layer
regressor.add(Dense(units=1))


#Compiling RNN

regressor.compile(optimizer='rmsprop', loss='mean_squared_error')

#Fitting to training set

regressor.fit(X_train, y_train, epochs=50, batch_size=32)




Epoch 1/50
5/5 [==============================] - 6s 117ms/step - loss: 0.1614
Epoch 2/50
5/5 [==============================] - 1s 116ms/step - loss: 0.0569
Epoch 3/50
5/5 [==============================] - 1s 138ms/step - loss: 0.0368
Epoch 4/50
5/5 [==============================] - 1s 144ms/step - loss: 0.0299
Epoch 5/50
5/5 [==============================] - 1s 188ms/step - loss: 0.0299
Epoch 6/50
5/5 [==============================] - 1s 183ms/step - loss: 0.0267
Epoch 7/50
5/5 [==============================] - 1s 156ms/step - loss: 0.0198
Epoch 8/50
5/5 [==============================] - 1s 155ms/step - loss: 0.0161
Epoch 9/50
5/5 [==============================] - 1s 179ms/step - loss: 0.0802
Epoch 10/50
5/5 [==============================] - 1s 232ms/step - loss: 0.0152
Epoch 11/50
5/5 [==============================] - 1s 170ms/step - loss: 0.0173
Epoch 12/50
5/5 [==============================] - 1s 190ms/step - loss: 0.0144
Epoch 13/50
5/5 [==============================] 

In [22]:
#Getting the testset like the trainig set
dataset_total = dataset["High"]
inputs = dataset_total[len(dataset_total) - len(testset) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

In [33]:
X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
predicted_stock = regressor.predict(X_test)
predicted_stock = np.reshape(predicted_stock, (predicted_stock.shape[0], predicted_stock.shape[1]))
predicted_stock = sc.inverse_transform(predicted_stock)

In [39]:
predicted_stock.shape

##There is something wrong with this. Have to spend some brainpower on this

(63, 60)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6f0cb4aa-1bc8-4a44-bd28-be65810b27e8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>